* analysis each different pollutant
    * why are they included? what are the effects? 
* focus on one area for each pollutant
* 1 hypothesis test using regression
* time series analysis
* 1 question to model

* pick region of interest 
* isolate data for region
* time series analysis and prediction graphs for individual pollutants

* graph of most and least polluted states in the US

- heat map of US aqi
- separate by region (northeast, southeast, midwest, west) and graph aqi over time 
- graph of defining parameters over time

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
from time_series_analysis import *
%matplotlib inline

In [5]:
all_data = pd.read_csv('final_merged_data.csv')

In [6]:
all_data.head()

,merge_column,state_name,county_name,date_local,o3,co,no2,so2,pm10,pm25,pressure,temperature,wind,aqi,category,defining_parameter
0,Alabama-Baldwin-2000-03-0100:00:00,Alabama,Baldwin,2000-03-01,0.036,nan,nan,nan,nan,nan,nan,nan,nan,43.000,Good,Ozone
1,Alabama-Baldwin-2000-03-0200:00:00,Alabama,Baldwin,2000-03-02,0.038,nan,nan,nan,nan,nan,nan,nan,nan,74.000,Moderate,Ozone
2,Alabama-Baldwin-2000-03-0300:00:00,Alabama,Baldwin,2000-03-03,0.035,nan,nan,nan,nan,nan,nan,nan,nan,34.000,Good,Ozone
3,Alabama-Baldwin-2000-03-0400:00:00,Alabama,Baldwin,2000-03-04,0.029,nan,nan,nan,nan,nan,nan,nan,nan,35.000,Good,Ozone
4,Alabama-Baldwin-2000-03-0500:00:00,Alabama,Baldwin,2000-03-05,0.036,nan,nan,nan,nan,nan,nan,nan,nan,54.000,Moderate,Ozone


In [7]:
all_data.isna().sum()/len(all_data)

merge_column         0.000
state_name           0.000
county_name          0.000
date_local           0.000
o3                   0.354
co                   0.789
no2                  0.751
so2                  0.664
pm10                 0.902
pm25                 0.754
pressure             0.815
temperature          0.614
wind                 0.696
aqi                  0.036
category             0.036
defining_parameter   0.036
dtype: float64

In [8]:
all_data.shape

(6589563, 16)

In [9]:
df_no_nulls = all_data.dropna(thresh=16)

In [10]:
df_no_nulls.shape

(41372, 16)

In [11]:
df_no_nulls.isna().sum()/len(df_no_nulls)

merge_column         0.000
state_name           0.000
county_name          0.000
date_local           0.000
o3                   0.000
co                   0.000
no2                  0.000
so2                  0.000
pm10                 0.000
pm25                 0.000
pressure             0.000
temperature          0.000
wind                 0.000
aqi                  0.000
category             0.000
defining_parameter   0.000
dtype: float64

In [12]:
df_no_nulls['defining_parameter'].value_counts()

PM2.5    17811
Ozone    17551
NO2       2483
PM10      2234
SO2       1231
CO          62
Name: defining_parameter, dtype: int64

In [13]:
df_no_nulls[df_no_nulls['defining_parameter'] == 'PM10']

,merge_column,state_name,county_name,date_local,o3,co,no2,so2,pm10,pm25,pressure,temperature,wind,aqi,category,defining_parameter
35162,Alabama-Jefferson-2014-01-1700:00:00,Alabama,Jefferson,2014-01-17,0.027,0.385,8.160,2.435,29.833,6.275,993.728,39.766,120.922,59.000,Moderate,PM10
35868,Alabama-Jefferson-2016-01-1000:00:00,Alabama,Jefferson,2016-01-10,0.023,0.277,5.054,0.509,28.000,2.267,993.975,36.388,140.533,26.000,Good,PM10
36527,Alabama-Jefferson-2018-01-2300:00:00,Alabama,Jefferson,2018-01-23,0.027,0.083,8.466,3.496,12.000,3.550,997.300,47.413,126.824,40.000,Good,PM10
86166,Alaska-FairbanksNorthStar-2014-07-1000:00:00,Alaska,Fairbanks North Star,2014-07-10,0.015,0.220,4.472,2.080,23.000,2.667,744.083,66.906,84.987,22.000,Good,PM10
86186,Alaska-FairbanksNorthStar-2014-07-3100:00:00,Alaska,Fairbanks North Star,2014-07-31,0.018,0.217,4.673,1.971,27.000,4.600,753.833,62.485,72.319,28.000,Good,PM10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243633,Wyoming-Sweetwater-2012-10-0300:00:00,Wyoming,Sweetwater,2012-10-03,0.033,0.209,2.522,0.569,45.714,5.867,826.631,49.253,100.947,60.000,Moderate,PM10
4243645,Wyoming-Sweetwater-2012-10-1500:00:00,Wyoming,Sweetwater,2012-10-15,0.034,0.200,2.932,0.328,17.571,4.167,824.646,48.700,126.258,47.000,Good,PM10
4243672,Wyoming-Sweetwater-2012-11-1100:00:00,Wyoming,Sweetwater,2012-11-11,0.034,0.200,1.527,0.109,15.087,4.467,822.989,14.094,140.640,42.000,Good,PM10
4243702,Wyoming-Sweetwater-2012-12-1100:00:00,Wyoming,Sweetwater,2012-12-11,0.032,0.100,1.766,0.230,11.000,3.000,818.366,24.347,126.415,39.000,Good,PM10


In [14]:
df = all_data.drop(['co','pm10'], axis=1)

In [15]:
df.isna().sum()/len(df)

merge_column         0.000
state_name           0.000
county_name          0.000
date_local           0.000
o3                   0.354
no2                  0.751
so2                  0.664
pm25                 0.754
pressure             0.815
temperature          0.614
wind                 0.696
aqi                  0.036
category             0.036
defining_parameter   0.036
dtype: float64

In [16]:
all_data.isna().sum()/len(all_data)

merge_column         0.000
state_name           0.000
county_name          0.000
date_local           0.000
o3                   0.354
co                   0.789
no2                  0.751
so2                  0.664
pm10                 0.902
pm25                 0.754
pressure             0.815
temperature          0.614
wind                 0.696
aqi                  0.036
category             0.036
defining_parameter   0.036
dtype: float64

In [17]:
all_data['date_local'] = pd.to_datetime(all_data['date_local'])

In [18]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6589563 entries, 0 to 6589562
Data columns (total 16 columns):
 #   Column              Dtype         
---  ------              -----         
 0   merge_column        object        
 1   state_name          object        
 2   county_name         object        
 3   date_local          datetime64[ns]
 4   o3                  float64       
 5   co                  float64       
 6   no2                 float64       
 7   so2                 float64       
 8   pm10                float64       
 9   pm25                float64       
 10  pressure            float64       
 11  temperature         float64       
 12  wind                float64       
 13  aqi                 float64       
 14  category            object        
 15  defining_parameter  object        
dtypes: datetime64[ns](1), float64(10), object(5)
memory usage: 804.4+ MB


In [19]:
all_data['year'] = pd.DatetimeIndex(all_data['date_local']).year
all_data['month'] = pd.DatetimeIndex(all_data['date_local']).month
all_data['day'] = pd.DatetimeIndex(all_data['date_local']).day

In [20]:
all_data.head()

,merge_column,state_name,county_name,date_local,o3,co,no2,so2,pm10,pm25,pressure,temperature,wind,aqi,category,defining_parameter,year,month,day
0,Alabama-Baldwin-2000-03-0100:00:00,Alabama,Baldwin,2000-03-01,0.036,nan,nan,nan,nan,nan,nan,nan,nan,43.000,Good,Ozone,2000,3,1
1,Alabama-Baldwin-2000-03-0200:00:00,Alabama,Baldwin,2000-03-02,0.038,nan,nan,nan,nan,nan,nan,nan,nan,74.000,Moderate,Ozone,2000,3,2
2,Alabama-Baldwin-2000-03-0300:00:00,Alabama,Baldwin,2000-03-03,0.035,nan,nan,nan,nan,nan,nan,nan,nan,34.000,Good,Ozone,2000,3,3
3,Alabama-Baldwin-2000-03-0400:00:00,Alabama,Baldwin,2000-03-04,0.029,nan,nan,nan,nan,nan,nan,nan,nan,35.000,Good,Ozone,2000,3,4
4,Alabama-Baldwin-2000-03-0500:00:00,Alabama,Baldwin,2000-03-05,0.036,nan,nan,nan,nan,nan,nan,nan,nan,54.000,Moderate,Ozone,2000,3,5


In [21]:
all_data.loc[all_data.aqi >=1000]

,merge_column,state_name,county_name,date_local,o3,co,no2,so2,pm10,pm25,pressure,temperature,wind,aqi,category,defining_parameter,year,month,day
133730,Arizona-Pinal-2007-05-0100:00:00,Arizona,Pinal,2007-05-01,0.045,nan,nan,3.000,479.167,nan,nan,nan,nan,2148.000,Hazardous,PM10,2007,5,1
133809,Arizona-Pinal-2007-07-1900:00:00,Arizona,Pinal,2007-07-19,0.043,nan,nan,3.000,1075.167,nan,nan,nan,nan,1734.000,Hazardous,PM10,2007,7,19
134536,Arizona-Pinal-2009-07-1700:00:00,Arizona,Pinal,2009-07-17,0.045,nan,nan,nan,751.167,nan,nan,nan,nan,1341.000,Hazardous,PM10,2009,7,17
134723,Arizona-Pinal-2010-01-2100:00:00,Arizona,Pinal,2010-01-21,0.034,nan,nan,nan,546.400,nan,nan,nan,nan,1657.000,Hazardous,PM10,2010,1,21
135252,Arizona-Pinal-2011-07-0500:00:00,Arizona,Pinal,2011-07-05,0.051,nan,nan,nan,906.833,67.200,nan,nan,nan,2212.000,Hazardous,PM10,2011,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5844659,California-Mono-2017-03-0400:00:00,California,Mono,2017-03-04,nan,nan,nan,nan,nan,nan,nan,36.333,99.597,1296.000,Hazardous,PM10,2017,3,4
5844681,California-Mono-2017-04-0600:00:00,California,Mono,2017-04-06,nan,nan,nan,nan,nan,nan,nan,44.583,110.616,3439.000,Hazardous,PM10,2017,4,6
5844793,California-Mono-2017-09-2000:00:00,California,Mono,2017-09-20,nan,nan,nan,nan,nan,nan,nan,53.312,109.067,1245.000,Hazardous,PM10,2017,9,20
5844813,California-Mono-2017-10-2000:00:00,California,Mono,2017-10-20,nan,nan,nan,nan,nan,nan,nan,39.875,120.754,1132.000,Hazardous,PM10,2017,10,20


In [22]:
print("Null values as a percent")
all_data.isna().sum()/len(all_data)

Null values as a percent


merge_column         0.000
state_name           0.000
county_name          0.000
date_local           0.000
o3                   0.354
co                   0.789
no2                  0.751
so2                  0.664
pm10                 0.902
pm25                 0.754
pressure             0.815
temperature          0.614
wind                 0.696
aqi                  0.036
category             0.036
defining_parameter   0.036
year                 0.000
month                0.000
day                  0.000
dtype: float64

In [26]:
all_data = all_data[all_data.state_name != 'Canada']
all_data = all_data[all_data.state_name != 'Country Of Mexico']
all_data = all_data[all_data.state_name != 'Virgin Islands']

In [22]:
va.head()

,state_name,county_name,date_local,o3,co,no2,so2,pm10,pm25,pressure,temperature,wind,aqi
3948506,Virginia,Albemarle,2008-03-31,0.019,nan,nan,nan,nan,nan,nan,nan,nan,nan
3948507,Virginia,Albemarle,2008-04-01,0.035,nan,nan,nan,nan,nan,nan,nan,nan,35.000
3948508,Virginia,Albemarle,2008-04-02,0.037,nan,nan,nan,nan,nan,nan,nan,nan,45.000
3948509,Virginia,Albemarle,2008-04-03,0.032,nan,nan,nan,nan,7.700,nan,nan,nan,32.000
3948510,Virginia,Albemarle,2008-04-04,0.027,nan,nan,nan,nan,nan,nan,nan,nan,30.000


## Visualizations

I want to see aqi levels by state on a map of the US. 

First I need to group my data by state and by year.

In [33]:
state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [34]:
all_data['state_initials'] = all_data['state_name'].map(state_abbrev)

In [35]:
all_data.head()

,merge_column,state_name,county_name,date_local,o3,co,no2,so2,pm10,pm25,pressure,temperature,wind,aqi,category,defining_parameter,year,month,day,state_initials
0,Alabama-Baldwin-2000-03-0100:00:00,Alabama,Baldwin,2000-03-01,0.036,nan,nan,nan,nan,nan,nan,nan,nan,43.000,Good,Ozone,2000,3,1,AL
1,Alabama-Baldwin-2000-03-0200:00:00,Alabama,Baldwin,2000-03-02,0.038,nan,nan,nan,nan,nan,nan,nan,nan,74.000,Moderate,Ozone,2000,3,2,AL
2,Alabama-Baldwin-2000-03-0300:00:00,Alabama,Baldwin,2000-03-03,0.035,nan,nan,nan,nan,nan,nan,nan,nan,34.000,Good,Ozone,2000,3,3,AL
3,Alabama-Baldwin-2000-03-0400:00:00,Alabama,Baldwin,2000-03-04,0.029,nan,nan,nan,nan,nan,nan,nan,nan,35.000,Good,Ozone,2000,3,4,AL
4,Alabama-Baldwin-2000-03-0500:00:00,Alabama,Baldwin,2000-03-05,0.036,nan,nan,nan,nan,nan,nan,nan,nan,54.000,Moderate,Ozone,2000,3,5,AL


In [52]:
annual_mean_aqi = pd.DataFrame()
for i in range(20):
    i += 2000
    year = all_data.loc[all_data.year == i]
    annual_mean_aqi[i] = year.groupby('state_initials')['aqi'].mean()

In [53]:
annual_mean_aqi

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
state_initials,,,,,,,,,,,,,,,,,,,,
AK,30.656,31.990,28.342,25.821,26.391,24.372,27.585,26.343,25.485,30.758,29.809,30.862,30.951,30.402,34.912,35.474,33.673,31.593,29.635,32.765
AL,56.976,48.593,47.511,47.148,46.797,48.147,50.588,53.226,46.884,41.289,48.808,45.810,43.267,37.897,39.937,40.805,41.175,39.237,38.321,39.713
AR,49.008,45.340,43.605,42.677,40.522,48.642,44.764,43.878,40.047,37.802,42.455,41.944,42.144,38.234,36.123,35.209,35.392,39.507,38.811,39.436
AZ,55.952,54.611,58.224,56.008,53.021,54.070,64.926,61.645,59.475,52.313,51.889,56.618,55.288,51.132,52.149,47.594,48.779,50.945,49.958,47.282
CA,61.828,63.768,65.665,64.049,60.121,56.357,60.004,58.137,60.562,56.287,54.111,58.224,57.134,55.929,56.233,55.918,55.074,57.472,57.038,46.329
CO,38.302,36.780,39.237,38.417,35.463,37.976,39.702,44.128,42.184,40.529,43.113,46.410,47.212,45.749,42.453,41.392,41.269,43.322,46.800,45.810
CT,48.387,55.563,58.274,51.782,47.203,51.428,47.929,49.992,45.689,39.781,43.839,43.158,47.997,48.162,44.813,45.512,44.310,41.294,41.232,41.299
DE,62.034,67.499,68.514,61.959,58.256,63.917,60.301,58.180,56.274,47.320,53.695,48.417,49.738,44.878,47.206,46.366,44.999,43.834,42.480,45.755
FL,48.422,43.770,41.209,41.641,44.101,44.504,46.079,44.420,41.846,39.295,42.120,41.247,38.209,36.936,37.925,36.078,38.094,38.437,36.973,39.901


In [67]:
# Initiate map with center around Montana
m = folium.Map(location=[48, -102], zoom_start=3)

In [69]:
# Add the color for the chloropleth:
m.choropleth(geo_data= ,
             name='choropleth',
             data=annual_mean_aqi, 
             columns=['state_initials', '2019'],
             key_on='feature.id',
             fill_color='YlGn',
             fill_opacity=0.7,
             line_opacity=0.2,
             legend_name='Air Quality Index')
folium.LayerControl().add_to(m)

TypeError: __init__() missing 1 required positional argument: 'geo_data'

In [66]:
# Save to html
m.save('test2.html')

In [57]:
# Add the color for the chloropleth:
m.choropleth(
 geo_data='us_counties.json',
 name='choropleth',
 data=annual_mean_aqi,
 columns=['state_initials', '2019'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Air Quality Index'
)
folium.LayerControl().add_to(m)

KeyError: "None of ['state_initials'] are in the columns"

# State Summary Data

In [63]:
states = all_data.groupby('state_name',sort=True)

In [44]:
state_data = all_data.groupby('state_name',sort=True).mean()

In [45]:
state_data.head()

,o3,co,no2,so2,pm10,pm25,pressure,temperature,wind,aqi
state_name,,,,,,,,,,
Alabama,0.030,0.526,8.955,2.132,19.980,11.906,998.543,57.892,96.029,45.467
Alaska,0.027,0.764,12.509,6.688,16.228,8.532,785.673,30.551,97.514,30.126
Arizona,0.040,0.445,13.449,2.565,33.738,7.571,856.099,63.322,93.027,53.860
Arkansas,0.031,0.479,9.748,1.916,19.628,11.158,nan,63.560,95.509,41.442
California,0.031,0.407,11.147,0.961,26.666,12.127,942.108,60.251,99.321,58.245


In [46]:
state_data.shape

(55, 10)

In [47]:
state_data

,o3,co,no2,so2,pm10,pm25,pressure,temperature,wind,aqi
state_name,,,,,,,,,,
Alabama,0.030,0.526,8.955,2.132,19.980,11.906,998.543,57.892,96.029,45.467
Alaska,0.027,0.764,12.509,6.688,16.228,8.532,785.673,30.551,97.514,30.126
Arizona,0.040,0.445,13.449,2.565,33.738,7.571,856.099,63.322,93.027,53.860
Arkansas,0.031,0.479,9.748,1.916,19.628,11.158,nan,63.560,95.509,41.442
California,0.031,0.407,11.147,0.961,26.666,12.127,942.108,60.251,99.321,58.245
Canada,0.029,nan,4.918,1.461,nan,6.663,nan,nan,nan,24.305
Colorado,0.038,0.451,11.708,1.686,22.809,7.412,806.175,49.800,96.347,41.924
Connecticut,0.034,0.453,12.803,1.933,19.457,9.777,999.208,51.057,104.294,46.872
Country Of Mexico,0.027,0.790,18.825,2.501,72.906,nan,nan,65.318,103.223,70.119


In [49]:
state_data.isna().sum()

o3             0
co             0
no2            1
so2            0
pm10           0
pm25           0
pressure       6
temperature    0
wind           0
aqi            0
dtype: int64

In [50]:
state_data.drop(['pressure'], axis=1, inplace=True)

In [57]:
#Nebraska has a null value for no2. I will replace it with the average of the no2 measurements in the states bordering Nebraska.
nebraska_border = ['Wyoming', 'South Dakota', 'Iowa', 
                   'Kansas', 'Colorado']
border_mean = state_data.loc[nebraska_border, 'no2'].mean()
state_data.fillna(border_mean, inplace=True)

In [58]:
state_data.isna().sum()

o3             0
co             0
no2            0
so2            0
pm10           0
pm25           0
temperature    0
wind           0
aqi            0
dtype: int64

In [59]:
state_data

,o3,co,no2,so2,pm10,pm25,temperature,wind,aqi
state_name,,,,,,,,,
Alabama,0.030,0.526,8.955,2.132,19.980,11.906,57.892,96.029,45.467
Alaska,0.027,0.764,12.509,6.688,16.228,8.532,30.551,97.514,30.126
Arizona,0.040,0.445,13.449,2.565,33.738,7.571,63.322,93.027,53.860
Arkansas,0.031,0.479,9.748,1.916,19.628,11.158,63.560,95.509,41.442
California,0.031,0.407,11.147,0.961,26.666,12.127,60.251,99.321,58.245
Colorado,0.038,0.451,11.708,1.686,22.809,7.412,49.800,96.347,41.924
Connecticut,0.034,0.453,12.803,1.933,19.457,9.777,51.057,104.294,46.872
Delaware,0.032,0.356,13.081,2.076,23.975,11.253,57.215,105.899,53.048
District Of Columbia,0.027,0.718,16.563,3.808,22.267,11.978,58.044,97.637,57.088
